In [ ]:
import numpy as np
import cv2
import math

# COLOR WEBCAM, REAL TIME 
# =======================

# ------------- PARAMS ----------------
# Memory length:
Size = 0        # 5 = Scan spatially +/- 5 pixels (0 to 10)
TimeMult = 2    # 2 = x2 more time depth than spatial width
InvCone = False # Cone can be wide at last frame and shrink, or the opposite:
pwr = 2.0       # Strength of the effect, 0 = None, 1 = a lot.
alpha = 0.4     # 0 = Original, 1 = Movement only
red = 4         # Resolution reduction (1, 2 or 4):
width = 640 // red
heigh = 480 // red
SaveVideo = True
# --------------------------------------
TimeSize = max(1, Size) * TimeMult
Size2 = TimeSize**2

# Open webcam:
cap = cv2.VideoCapture(0)
ret = cap.set(3, width)
ret = cap.set(4, heigh)

# Create an initial list of 10 frames:
ret, frame = cap.read()
img = []
for i in range(TimeSize):
    img.append(cv2.copyMakeBorder(frame,0,0,0,0,cv2.BORDER_REPLICATE))

# Initialize output images:
gray =   cv2.copyMakeBorder(frame,0,0,0,0,cv2.BORDER_REPLICATE)  # BGR
mask =   cv2.copyMakeBorder(frame,0,0,0,0,cv2.BORDER_REPLICATE)  # BGR
triple = cv2.copyMakeBorder(frame,0,0,0,width*2,cv2.BORDER_REPLICATE)

# Prepare video output file:
if SaveVideo:
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output_RT.avi',fourcc, 1.0, (width*3, heigh), True)

# Std dev max value found, updated at every frame:
MaxStd = np.repeat(1.0, gray[1,1].shape[0])

while(True):
    
    # Lower MaxStd so it auto-adapts filter sensitivity:
    MaxStd = MaxStd * 0.8
    
    #for i in range(Size * TimeMult):
        # Read a new frame and convert:
        #ret, frame = cap.read()
        #img[i] = frame # cv2.copyMakeBorder(frame,0,0,0,0,cv2.BORDER_REPLICATE)  # BGR        
        # Add noise:
        #img[i] = np.clip(np.add(img[i], np.random.normal(0, 15, (gray.shape))), 0, 255)
        
    # Add a new image to list, keep count the same:
    ret, frame = cap.read()
    img.insert(0, cv2.copyMakeBorder(frame,0,0,0,0,cv2.BORDER_REPLICATE))
    del img[-1]
    
    # Filter last image, pixel by pixel:
    for x in range(1,gray.shape[0]):        
        for y in range(1,gray.shape[1]):
            
            num = 0
            avg = np.repeat(0.0, gray[1,1].shape[0])
            for i in range(TimeSize):
                dt2 = (i if InvCone else TimeSize-i)**2
                for x1 in range(x-Size, x+Size+1):
                    x2 = max(0, min(gray.shape[0]-1, x1))
                    dx2 = ((x-x2)/TimeMult)**2
                    for y1 in range(max(1,y-Size), 1+min(gray.shape[1], y+Size)):
                        y2 = max(0, min(gray.shape[1]-1, y1))
                        dy2 = ((y-y2)/TimeMult)**2
                        if ((dt2 + dx2 + dy2) <= Size2):
                            num = num + 1
                            avg = avg + img[i][x1,y1]
            avg = avg / num            
            std = np.repeat(0.0, gray[1,1].shape[0])            
            for i in range(TimeSize):
                dt2 = (i if InvCone else TimeSize-i)**2
                for x1 in range(x-Size, x+Size+1):
                    x2 = max(0, min(gray.shape[0]-1, x1))
                    dx2 = ((x-x2)/TimeMult)**2
                    for y1 in range(max(1,y-Size), 1+min(gray.shape[1], y+Size)):
                        y2 = max(0, min(gray.shape[1]-1, y1))
                        dy2 = ((y-y2)/TimeMult)**2
                        if ((dt2 + dx2 + dy2) <= Size2):
                            std = std + np.square(img[i][x,y] - avg)
            std = np.sqrt(std / (num-1.0))
            MaxStd = np.maximum(MaxStd, std)
            
            # Real image pixel value:
            value = img[0][x,y]

            # Relativize coef for this pixel:
            coef = (value - avg) / MaxStd
            
            for c in range(0, coef.shape[0]):
                if (coef[c] > 0):
                    coef[c] = 1.0 + np.log(1.0 + coef[c])
                else:
                    coef[c] = np.exp(coef[c])
            
            # Define pixel color in both output images:
            gray[x,y] = np.clip(value * np.power(coef, pwr), 0, 255)  # BGR
            #gray[x,y] = value*(1-alpha) + alpha * np.clip((126 * np.power(coef, pwr)), 0, 255)
            mask[x,y] = np.clip(126 * np.power(coef, pwr), 0, 255)
    
    # Show original, coefs and then filtered:
    triple[1:gray.shape[0], 1:gray.shape[1]] = img[1][1:gray.shape[0], 1:gray.shape[1]]
    triple[1:gray.shape[0], gray.shape[1]+1:gray.shape[1]*2] = mask[1:gray.shape[0], 1:gray.shape[1]]   
    triple[1:gray.shape[0], gray.shape[1]*2+1:gray.shape[1]*3] = gray[1:gray.shape[0], 1:gray.shape[1]]
    
    # Show final composite image:
    cv2.imshow('Relativize filter', triple)    
    # write the frame to video:
    if SaveVideo:
        out.write(triple)

    # Press q to exit:    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
# If video doesnt show, manually reset the kernel to free the file:
if SaveVideo:
    out.release

In [ ]:
TimeSize


In [ ]:
len(img)
